In [ ]:
# EXAM 2023 ANSWER

#Question 1 – Détection, classification et localisation de crayons colorés
#1.a Détection des pointes de crayons
#i. Lecture et affichage de l’image RGB
import cv2
import numpy as np
import matplotlib.pyplot as plt


img = cv2.imread('Pencils.jpg')
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


plt.imshow(img_rgb)
plt.title('Image RGB – Crayons')
plt.axis('off')
plt.show()

# ii. Affichage des canaux RGB
channels = ['Rouge','Vert','Bleu']
plt.figure(figsize=(12,4))
for i in range(3):
plt.subplot(1,3,i+1)
plt.imshow(img_rgb[:,:,i], cmap='gray')
plt.title(channels[i])
plt.axis('off')
plt.show()

#iii. Choix du canal
#Le canal bleu offre le meilleur contraste entre les pointes (blanches et bleues) et le fond, avec peu de faux positifs du fond.

#iv. Seuillage d’Otsu
blue = img_rgb[:,:,2]
_, binary = cv2.threshold(blue, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


plt.imshow(binary, cmap='gray')
plt.title('Image binaire – Otsu')
plt.axis('off')
plt.show()

# v. Commentaire
#Otsu segmente correctement la majorité des pointes, mais certaines pointes bleues sombres sont partiellement segmentées en raison d’un contraste plus faible.

#1.b Distinction entre pointes blanches et bleues
#i. Histogramme du canal rouge
red = img_rgb[:,:,0]
values = red[binary==1]


plt.hist(values, bins=256, range=(0,256))
plt.xlabel('Intensité du canal rouge')
plt.ylabel('Nombre de pixels')
plt.title('Histogramme du canal rouge (zones des pointes)')
plt.show()

# ii. Séparation par seuillage
#Un seuil intermédiaire (ex. 150) sépare efficacement les pointes blanches (intensité élevée) des pointes bleues.
th = 150
white_tips = np.logical_and(binary==1, red > th).astype(np.uint8)
blue_tips = np.logical_and(binary==1, red <= th).astype(np.uint8)

#iii. Morphologie
#Ouverture + fermeture avec un élément structurant elliptique 5×5.
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
white_clean = cv2.morphologyEx(white_tips, cv2.MORPH_CLOSE, kernel)
blue_clean = cv2.morphologyEx(blue_tips, cv2.MORPH_CLOSE, kernel)

#1.c Localisation par centroides
#i. Extraction des centroides
_, _, _, centroids_w = cv2.connectedComponentsWithStats(white_clean)
_, _, _, centroids_b = cv2.connectedComponentsWithStats(blue_clean)

#ii. Visualisation
out = img_rgb.copy()
for c in centroids_w[1:]:
cv2.circle(out, (int(c[0]),int(c[1])), 5, (0,0,0), -1)
for c in centroids_b[1:]:
cv2.circle(out, (int(c[0]),int(c[1])), 5, (0,255,0), -1)

plt.imshow(out)
plt.title('Centroides des pointes')
plt.axis('off')
plt.show()

#Question 2 – Détection de véhicules et infractions
#2.a Détection de voitures par Haar Cascade
cars = cv2.imread('Cars.jpg')
cars_rgb = cv2.cvtColor(cars, cv2.COLOR_BGR2RGB)

cascade = cv2.CascadeClassifier('cars.xml')
detections = cascade.detectMultiScale(cars, scaleFactor=1.01)

out = cars_rgb.copy()
for (x,y,w,h) in detections:
cv2.rectangle(out,(x,y),(x+w,y+h),(0,255,0),2)
plt.imshow(out)
plt.axis('off')
plt.show()
#Commentaire : beaucoup de faux positifs dus à un scaleFactor trop faible.
#Reglage des parametres : detections = cascade.detectMultiScale(cars, scaleFactor=1.2, minNeighbors=5, minSize=(30,30))

#2.b Détection du marquage BUS (Template Matching)
template = cv2.imread('Template.jpg')
template_rgb = cv2.cvtColor(template, cv2.COLOR_BGR2RGB)

res = cv2.matchTemplate(cars_rgb, template_rgb, cv2.TM_SQDIFF)
min_val, _, min_loc, _ = cv2.minMaxLoc(res)

#2.c Détection d’infractions
for (x,y,w,h) in detections:
if abs(x - min_loc[0]) < 150:
cv2.rectangle(out,(x,y),(x+w,y+h),(255,0,0),2)

#Question 3 – Réseaux de neurones
#3.a CNN MNIST
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train,x_test = x_train/255.0, x_test/255.0
x_train = x_train[...,None]
x_test = x_test[...,None]
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

model = Sequential([
Conv2D(6,(3,3),activation='relu',input_shape=(28,28,1)),
MaxPooling2D((2,2)),
Conv2D(16,(3,3),activation='relu'),
MaxPooling2D((2,2)),
Flatten(),
Dense(120,activation='relu'),
Dense(84,activation='relu'),
Dense(10,activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs=2,batch_size=32,validation_data=(x_test,y_test))

#iv–v Discussion
#   Learning rate trop élevé : divergence
#   Trop faible : convergence lente
#   Underfitting : faible accuracy entraînement et validation → modèle trop simple

#3.b InceptionV3
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
model = InceptionV3(weights='imagenet')

def prep(path):
img = cv2.imread(path)
img = cv2.resize(img,(299,299))
img = preprocess_input(img.astype(np.float32))
return np.expand_dims(img,0)

pred = model.predict(prep('Test1.jpg'))
decode_predictions(pred, top=3)
#Discussion : variations d’angle, d’éclairage et d’occlusion expliquent la baisse de performance.